<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey%20Model%20Mark%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
#Import Even-Strength On-Ice Rates Data
skater_EV_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Rates.csv')
skater_EV_rates_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,46.28,2.69,1.89,0.05,9.22,12.50,14.78,0.80,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,53.16,53.30,53.14,...,50.62,2.49,2.50,0.59,4.00,5.43,6.79,-0.01,10.03,90.59
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,54.51,53.65,53.97,...,45.84,2.52,2.05,0.64,5.17,5.64,7.91,0.47,7.68,93.24
3,Aaron Ekblad,21-22,FLA,D,61,1092.75,62.97,58.86,59.78,59.32,...,47.19,3.39,2.31,1.60,11.75,16.84,21.62,1.08,9.96,91.62
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,51.47,49.69,49.55,...,56.61,2.24,2.16,-0.06,1.78,-0.54,-1.01,0.08,8.39,90.90


In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_rates = skater_EV_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF/60','GA/60','xGF/60','xGA/60']]
skater_EV_rates

,Player,Season,Team,Position,GP,TOI,GF/60,GA/60,xGF/60,xGA/60
0,A.J. Greer,21-22,N.J,L,9,70.45,2.56,2.51,2.69,1.89
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,3.38,2.79,2.49,2.50
2,Aaron Ekblad,20-21,FLA,D,35,630.15,2.40,1.76,2.52,2.05
3,Aaron Ekblad,21-22,FLA,D,61,1092.75,3.89,2.29,3.39,2.31
4,Aaron Ness,19-20,ARI,D,24,292.63,2.63,2.69,2.24,2.16
...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,2.93,1.78,2.16,2.13
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.57,2.42,2.10,1.95
2477,Zdeno Chara,21-22,NYI,D,72,1196.30,2.43,2.13,2.36,2.77
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,1.75,2.08,1.94,1.98


In [4]:
skater_EV_rates['EV_TOI'] = skater_EV_rates['TOI']*3600
skater_EV_rates

,Player,Season,Team,Position,GP,TOI,GF/60,GA/60,xGF/60,xGA/60,EV_TOI
0,A.J. Greer,21-22,N.J,L,9,70.45,2.56,2.51,2.69,1.89,253620.0
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,3.38,2.79,2.49,2.50,4565808.0
2,Aaron Ekblad,20-21,FLA,D,35,630.15,2.40,1.76,2.52,2.05,2268540.0
3,Aaron Ekblad,21-22,FLA,D,61,1092.75,3.89,2.29,3.39,2.31,3933900.0
4,Aaron Ness,19-20,ARI,D,24,292.63,2.63,2.69,2.24,2.16,1053468.0
...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,2.93,1.78,2.16,2.13,4216320.0
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.57,2.42,2.10,1.95,2942712.0
2477,Zdeno Chara,21-22,NYI,D,72,1196.30,2.43,2.13,2.36,2.77,4306680.0
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,1.75,2.08,1.94,1.98,2827620.0


In [5]:
#Convert rate/60 to rates/s
skater_EV_rates['GF_EV'] = skater_EV_rates['GF/60']/3600
skater_EV_rates['GA_EV'] = skater_EV_rates['GA/60']/3600
skater_EV_rates['xGF_EV'] = skater_EV_rates['xGF/60']/3600
skater_EV_rates['xGA_EV'] = skater_EV_rates['xGA/60']/3600
skater_EV_rates

,Player,Season,Team,Position,GP,TOI,GF/60,GA/60,xGF/60,xGA/60,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV
0,A.J. Greer,21-22,N.J,L,9,70.45,2.56,2.51,2.69,1.89,253620.0,0.000711,0.000697,0.000747,0.000525
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,3.38,2.79,2.49,2.50,4565808.0,0.000939,0.000775,0.000692,0.000694
2,Aaron Ekblad,20-21,FLA,D,35,630.15,2.40,1.76,2.52,2.05,2268540.0,0.000667,0.000489,0.000700,0.000569
3,Aaron Ekblad,21-22,FLA,D,61,1092.75,3.89,2.29,3.39,2.31,3933900.0,0.001081,0.000636,0.000942,0.000642
4,Aaron Ness,19-20,ARI,D,24,292.63,2.63,2.69,2.24,2.16,1053468.0,0.000731,0.000747,0.000622,0.000600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,2.93,1.78,2.16,2.13,4216320.0,0.000814,0.000494,0.000600,0.000592
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.57,2.42,2.10,1.95,2942712.0,0.000714,0.000672,0.000583,0.000542
2477,Zdeno Chara,21-22,NYI,D,72,1196.30,2.43,2.13,2.36,2.77,4306680.0,0.000675,0.000592,0.000656,0.000769
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,1.75,2.08,1.94,1.98,2827620.0,0.000486,0.000578,0.000539,0.000550


In [6]:
#Drop TOI, GF/60, GA/60, xGF/60 and xGA/60 columns
skater_EV_rates = skater_EV_rates.drop(columns = ['TOI','GF/60','GA/60','xGF/60','xGA/60'])
skater_EV_rates

,Player,Season,Team,Position,GP,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600
...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550


In [7]:
#Rename columns
skater_EV_rates = skater_EV_rates.rename(columns = {'Position':'Position_EV','GP':'GP_EV'})
skater_EV_rates

,Player,Season,Team,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600
...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550


In [8]:
#Import PP On-Ice Rates Data
skater_PP_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/PP%20On-Ice%20Rates.csv')
skater_PP_rates_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,81.87,84.52,85.63,...,14.08,6.14,0.45,5.91,39.94,54.16,69.80,5.69,12.57,95.21
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,87.33,87.48,87.89,...,15.16,8.24,0.69,8.92,60.02,79.28,94.85,7.55,13.94,91.44
2,Aaron Ekblad,21-22,FLA,D,61,213.88,92.51,86.16,86.78,87.39,...,14.02,8.23,0.74,9.14,48.17,65.00,83.10,7.49,17.33,91.27
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,81.06,84.35,86.69,...,13.18,6.14,0.63,4.42,36.83,52.28,72.63,5.52,10.62,93.92
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,75.97,79.96,82.55,...,16.41,5.95,1.69,5.32,28.65,42.88,61.26,4.25,16.26,88.74


In [9]:
#Reduce PP On-Ice Totals Data
skater_PP_rates = skater_PP_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
skater_PP_rates

,Player,Season,Team,Position,GP,TOI,G±/60
0,Aaron Ekblad,19-20,FLA,D,67,95.90,5.91
1,Aaron Ekblad,20-21,FLA,D,35,132.75,8.92
2,Aaron Ekblad,21-22,FLA,D,61,213.88,9.14
3,Adam Boqvist,19-20,CHI,D,41,87.02,4.42
4,Adam Boqvist,20-21,CHI,D,35,118.65,5.32
...,...,...,...,...,...,...,...
1279,Zach Werenski,19-20,CBJ,D,63,171.17,4.68
1280,Zach Werenski,20-21,CBJ,D,33,73.60,4.07
1281,Zach Werenski,21-22,CBJ,D,64,215.22,6.07
1282,Zack Kassian,19-20,EDM,R,58,23.18,0.49


In [10]:
#Convert G±/60 to G/s and TOI to PP_TOI(s)
skater_PP_rates['G±_PP'] = skater_PP_rates['G±/60']/3600
skater_PP_rates['PP_TOI'] = skater_PP_rates['TOI']*3600
skater_PP_rates

,Player,Season,Team,Position,GP,TOI,G±/60,G±_PP,PP_TOI
0,Aaron Ekblad,19-20,FLA,D,67,95.90,5.91,0.001642,345240.0
1,Aaron Ekblad,20-21,FLA,D,35,132.75,8.92,0.002478,477900.0
2,Aaron Ekblad,21-22,FLA,D,61,213.88,9.14,0.002539,769968.0
3,Adam Boqvist,19-20,CHI,D,41,87.02,4.42,0.001228,313272.0
4,Adam Boqvist,20-21,CHI,D,35,118.65,5.32,0.001478,427140.0
...,...,...,...,...,...,...,...,...,...
1279,Zach Werenski,19-20,CBJ,D,63,171.17,4.68,0.001300,616212.0
1280,Zach Werenski,20-21,CBJ,D,33,73.60,4.07,0.001131,264960.0
1281,Zach Werenski,21-22,CBJ,D,64,215.22,6.07,0.001686,774792.0
1282,Zack Kassian,19-20,EDM,R,58,23.18,0.49,0.000136,83448.0


In [11]:
#Drop columns
skater_PP_rates = skater_PP_rates.drop(columns = ['TOI','G±/60'])
skater_PP_rates

,Player,Season,Team,Position,GP,G±_PP,PP_TOI
0,Aaron Ekblad,19-20,FLA,D,67,0.001642,345240.0
1,Aaron Ekblad,20-21,FLA,D,35,0.002478,477900.0
2,Aaron Ekblad,21-22,FLA,D,61,0.002539,769968.0
3,Adam Boqvist,19-20,CHI,D,41,0.001228,313272.0
4,Adam Boqvist,20-21,CHI,D,35,0.001478,427140.0
...,...,...,...,...,...,...,...
1279,Zach Werenski,19-20,CBJ,D,63,0.001300,616212.0
1280,Zach Werenski,20-21,CBJ,D,33,0.001131,264960.0
1281,Zach Werenski,21-22,CBJ,D,64,0.001686,774792.0
1282,Zack Kassian,19-20,EDM,R,58,0.000136,83448.0


In [12]:
#Rename columns
skater_PP_rates = skater_PP_rates.rename(columns = {'Position':'Position_PP','GP':'GP_PP'})
skater_PP_rates

,Player,Season,Team,Position_PP,GP_PP,G±_PP,PP_TOI
0,Aaron Ekblad,19-20,FLA,D,67,0.001642,345240.0
1,Aaron Ekblad,20-21,FLA,D,35,0.002478,477900.0
2,Aaron Ekblad,21-22,FLA,D,61,0.002539,769968.0
3,Adam Boqvist,19-20,CHI,D,41,0.001228,313272.0
4,Adam Boqvist,20-21,CHI,D,35,0.001478,427140.0
...,...,...,...,...,...,...,...
1279,Zach Werenski,19-20,CBJ,D,63,0.001300,616212.0
1280,Zach Werenski,20-21,CBJ,D,33,0.001131,264960.0
1281,Zach Werenski,21-22,CBJ,D,64,0.001686,774792.0
1282,Zack Kassian,19-20,EDM,R,58,0.000136,83448.0


In [13]:
#Import SH On-Ice Totals Data
skater_SH_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/SH%20On-Ice%20Rates.csv')
skater_SH_rates_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA/60,xGF/60,xGA/60,G±/60,S±/60,F±/60,C±/60,xG±/60,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,22.51,21.41,19.08,...,101.33,1.72,8.54,-10.17,-43.66,-57.60,-77.44,-6.82,4.98,82.03
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,17.92,14.84,15.32,...,91.36,0.90,6.94,-4.86,-44.75,-65.14,-74.83,-6.04,11.15,89.08
2,Aaron Ekblad,21-22,FLA,D,61,167.00,25.76,19.05,17.45,16.22,...,92.41,1.64,6.87,-4.06,-43.86,-56.93,-74.52,-5.23,15.97,89.17
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,20.18,17.25,15.49,...,100.31,1.06,7.38,-4.47,-37.59,-60.70,-81.93,-6.32,10.26,88.52
4,Adam Fox,21-22,NYR,D,78,163.83,17.42,11.66,11.33,9.96,...,101.10,0.60,7.46,-4.19,-49.91,-68.53,-89.91,-6.85,14.77,90.76


In [14]:
#Reduce SH On-Ice Totals Data
skater_SH_rates = skater_SH_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]

skater_SH_rates

,Player,Season,Team,Position,GP,TOI,G±/60
0,Aaron Ekblad,19-20,FLA,D,65,134.87,-10.17
1,Aaron Ekblad,20-21,FLA,D,33,86.47,-4.86
2,Aaron Ekblad,21-22,FLA,D,61,167.00,-4.06
3,Adam Fox,20-21,NYR,D,54,139.75,-4.47
4,Adam Fox,21-22,NYR,D,78,163.83,-4.19
...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,214.07,-5.95
1280,Zdeno Chara,20-21,WSH,D,55,145.72,-6.65
1281,Zdeno Chara,21-22,NYI,D,68,123.48,-2.72
1282,Zemgus Girgensons,19-20,BUF,C,68,147.87,-9.21


In [15]:
#Convert TOI to TOI(s) and G±/60 to G± (s)
skater_SH_rates['G±_SH'] = skater_SH_rates['G±/60']/3600
skater_SH_rates['SH_TOI'] = skater_SH_rates['TOI']*3600
skater_SH_rates

,Player,Season,Team,Position,GP,TOI,G±/60,G±_SH,SH_TOI
0,Aaron Ekblad,19-20,FLA,D,65,134.87,-10.17,-0.002825,485532.0
1,Aaron Ekblad,20-21,FLA,D,33,86.47,-4.86,-0.001350,311292.0
2,Aaron Ekblad,21-22,FLA,D,61,167.00,-4.06,-0.001128,601200.0
3,Adam Fox,20-21,NYR,D,54,139.75,-4.47,-0.001242,503100.0
4,Adam Fox,21-22,NYR,D,78,163.83,-4.19,-0.001164,589788.0
...,...,...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,214.07,-5.95,-0.001653,770652.0
1280,Zdeno Chara,20-21,WSH,D,55,145.72,-6.65,-0.001847,524592.0
1281,Zdeno Chara,21-22,NYI,D,68,123.48,-2.72,-0.000756,444528.0
1282,Zemgus Girgensons,19-20,BUF,C,68,147.87,-9.21,-0.002558,532332.0


In [16]:
#Drop columns
skater_SH_rates = skater_SH_rates.drop(columns = ['G±/60','TOI'])
skater_SH_rates

,Player,Season,Team,Position,GP,G±_SH,SH_TOI
0,Aaron Ekblad,19-20,FLA,D,65,-0.002825,485532.0
1,Aaron Ekblad,20-21,FLA,D,33,-0.001350,311292.0
2,Aaron Ekblad,21-22,FLA,D,61,-0.001128,601200.0
3,Adam Fox,20-21,NYR,D,54,-0.001242,503100.0
4,Adam Fox,21-22,NYR,D,78,-0.001164,589788.0
...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,-0.001653,770652.0
1280,Zdeno Chara,20-21,WSH,D,55,-0.001847,524592.0
1281,Zdeno Chara,21-22,NYI,D,68,-0.000756,444528.0
1282,Zemgus Girgensons,19-20,BUF,C,68,-0.002558,532332.0


In [17]:
#Rename columns
skater_SH_rates = skater_SH_rates.rename(columns = {'Position':'Position_SH','GP':'GP_SH'})
skater_SH_rates

,Player,Season,Team,Position_SH,GP_SH,G±_SH,SH_TOI
0,Aaron Ekblad,19-20,FLA,D,65,-0.002825,485532.0
1,Aaron Ekblad,20-21,FLA,D,33,-0.001350,311292.0
2,Aaron Ekblad,21-22,FLA,D,61,-0.001128,601200.0
3,Adam Fox,20-21,NYR,D,54,-0.001242,503100.0
4,Adam Fox,21-22,NYR,D,78,-0.001164,589788.0
...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,-0.001653,770652.0
1280,Zdeno Chara,20-21,WSH,D,55,-0.001847,524592.0
1281,Zdeno Chara,21-22,NYI,D,68,-0.000756,444528.0
1282,Zemgus Girgensons,19-20,BUF,C,68,-0.002558,532332.0


In [18]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
skater_box_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,...,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,...,20,11,1,6,0,0,5,2,1,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,...,34,56,14,10,0,0,-4,0,0,0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,...,24,46,14,8,1,1,-6,0,0,0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,...,62,54,13,9,0,0,-4,0,0,0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,...,17,49,0,5,0,0,5,0,0,0


In [19]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2']]
skater_box_totals.head()

,Player,Season,Team,Position,GP,TOI,G,A1,A2
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0


In [20]:
skater_box_totals['TOI_tot'] = skater_box_totals['TOI']*3600
skater_box_totals

,Player,Season,Team,Position,GP,TOI,G,A1,A2,TOI_tot
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,254520.0
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,5536512.0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,3161448.0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,5470920.0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1082988.0
...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,1428.48,5,2,7,5142528.0
2493,Zdeno Chara,20-21,WSH,D,55,1006.27,2,4,4,3622572.0
2494,Zdeno Chara,21-22,NYI,D,72,1348.45,2,8,4,4854420.0
2495,Zemgus Girgensons,19-20,BUF,C,69,951.77,12,4,3,3426372.0


In [21]:
skater_box_totals = skater_box_totals.drop(columns = {'TOI'})
skater_box_totals

,Player,Season,Team,Position,GP,G,A1,A2,TOI_tot
0,A.J. Greer,21-22,N.J,L,9,1,0,1,254520.0
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,5536512.0
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,3161448.0
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,5470920.0
4,Aaron Ness,19-20,ARI,D,24,0,1,0,1082988.0
...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,5,2,7,5142528.0
2493,Zdeno Chara,20-21,WSH,D,55,2,4,4,3622572.0
2494,Zdeno Chara,21-22,NYI,D,72,2,8,4,4854420.0
2495,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,3426372.0


In [22]:
#Look at positions to get correct F/D average points
#forwards_table = skater_box_totals.loc[skater_box_totals.Position!= "D"]
#defense_table = skater_box_totals.loc[skater_box_totals.Position== "D"]

In [23]:
#Calculate Mean Points of forwards and defensemen
#Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)
#Defense_mean = defense_table['Points'].mean()
#print(Defensemen_mean)

In [24]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine
#pd.options.mode.chained_assignment = None
#forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
#forwards_table.head(10)

#defense_table['PAA'] = defense_table['Points'] - Defense_mean
#defensemen_table.head(10)

#Add dataframes back together and sort again
#concat_frames = [forwards_table, defense_table]
#skater_box_totals = pd.concat(concat_frames)
#skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

In [25]:
#Combine skater EV and PP stats
skater_EV_PP = pd.merge(skater_EV_rates, skater_PP_rates, how="outer", on=['Player','Season'])
skater_EV_PP

,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,Team_y,Position_PP,GP_PP,G±_PP,PP_TOI
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,FLA,D,67.0,0.001642,345240.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,FLA,D,35.0,0.002478,477900.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,FLA,D,61.0,0.002539,769968.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,NaN,NaN,NaN,NaN,NaN
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,NaN,NaN,NaN,NaN,NaN
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,NaN,NaN,NaN,NaN,NaN
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,NaN,NaN,NaN,NaN,NaN


In [26]:
#Drop duplicate columns
skater_EV_PP = skater_EV_PP.drop(columns = ['Team_y','Position_PP','GP_PP'])
skater_EV_PP

,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,G±_PP,PP_TOI
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,0.001642,345240.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,0.002478,477900.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,0.002539,769968.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,NaN,NaN
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,NaN,NaN
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,NaN,NaN
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,NaN,NaN


In [27]:
#Merge EV,PP,SH tables together
skater_EV_PP_SH = pd.merge(skater_EV_PP, skater_SH_rates, how="left", on=["Player", "Season"])
skater_EV_PP_SH

,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,G±_PP,PP_TOI,Team,Position_SH,GP_SH,G±_SH,SH_TOI
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,0.001642,345240.0,FLA,D,65.0,-0.002825,485532.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,0.002478,477900.0,FLA,D,33.0,-0.001350,311292.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,0.002539,769968.0,FLA,D,61.0,-0.001128,601200.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,NaN,NaN,BOS,D,67.0,-0.001653,770652.0
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,NaN,NaN,WSH,D,55.0,-0.001847,524592.0
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,NaN,NaN,NYI,D,68.0,-0.000756,444528.0
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,NaN,NaN,BUF,C,68.0,-0.002558,532332.0


In [28]:
#Clean up data
skater_EV_PP_SH = skater_EV_PP_SH.drop(columns = ['Team','Position_SH','GP_SH'])
skater_EV_PP_SH

,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,G±_PP,PP_TOI,G±_SH,SH_TOI
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,0.001642,345240.0,-0.002825,485532.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,0.002478,477900.0,-0.001350,311292.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,0.002539,769968.0,-0.001128,601200.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,NaN,NaN,-0.001653,770652.0
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,NaN,NaN,-0.001847,524592.0
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,NaN,NaN,-0.000756,444528.0
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,NaN,NaN,-0.002558,532332.0


In [29]:
#Merge EV,PP,SH and Points tables together
full_skater_totals = pd.merge(skater_EV_PP_SH, skater_box_totals, how="left", on=["Player", "Season"])
full_skater_totals.head()

,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,PP_TOI,G±_SH,SH_TOI,Team,Position,GP,G,A1,A2,TOI_tot
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,...,NaN,NaN,NaN,N.J,L,9,1,0,1,254520.0
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,...,345240.0,-0.002825,485532.0,FLA,D,67,5,23,13,5536512.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,...,477900.0,-0.001350,311292.0,FLA,D,35,11,5,6,3161448.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,...,769968.0,-0.001128,601200.0,FLA,D,61,15,13,29,5470920.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,...,NaN,NaN,NaN,ARI,D,24,0,1,0,1082988.0


In [30]:
#Replace NaN with 0
full_skater_totals = full_skater_totals.replace(np.NaN, 0)
full_skater_totals

,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,PP_TOI,G±_SH,SH_TOI,Team,Position,GP,G,A1,A2,TOI_tot
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,...,0.0,0.000000,0.0,N.J,L,9,1,0,1,254520.0
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,...,345240.0,-0.002825,485532.0,FLA,D,67,5,23,13,5536512.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,...,477900.0,-0.001350,311292.0,FLA,D,35,11,5,6,3161448.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,...,769968.0,-0.001128,601200.0,FLA,D,61,15,13,29,5470920.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,...,0.0,0.000000,0.0,ARI,D,24,0,1,0,1082988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,...,0.0,-0.001653,770652.0,BOS,D,68,5,2,7,5142528.0
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,...,0.0,-0.001847,524592.0,WSH,D,55,2,4,4,3622572.0
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,...,0.0,-0.000756,444528.0,NYI,D,72,2,8,4,4854420.0
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,...,0.0,-0.002558,532332.0,BUF,C,69,12,4,3,3426372.0


In [31]:
#Drop Repeated Columns
full_skater_totals = full_skater_totals.drop(columns = ['Team','Position','GP'])
full_skater_totals.head(10)


,Player,Season,Team_x,Position_EV,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,G±_PP,PP_TOI,G±_SH,SH_TOI,G,A1,A2,TOI_tot
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,0.000000,0.0,0.000000,0.0,1,0,1,254520.0
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,0.001642,345240.0,-0.002825,485532.0,5,23,13,5536512.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,0.002478,477900.0,-0.001350,311292.0,11,5,6,3161448.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,0.002539,769968.0,-0.001128,601200.0,15,13,29,5470920.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,0.000000,0.0,0.000000,0.0,0,1,0,1082988.0
5,Adam Boqvist,19-20,CHI,D,41,2030220.0,0.000919,0.000825,0.000764,0.000819,0.001228,313272.0,0.000000,0.0,4,5,4,2394432.0
6,Adam Boqvist,20-21,CHI,D,35,1655892.0,0.000719,0.000803,0.000661,0.000775,0.001478,427140.0,0.000000,0.0,2,2,12,2140740.0
7,Adam Boqvist,21-22,CBJ,D,52,2748780.0,0.000894,0.000967,0.000672,0.000836,0.001339,356832.0,0.000000,0.0,11,4,7,3189780.0
8,Adam Brooks,19-20,TOR,C,7,185760.0,0.000950,0.000319,0.000767,0.000653,0.000000,0.0,0.000000,0.0,0,2,1,197892.0
9,Adam Brooks,20-21,TOR,C,11,397368.0,0.000775,0.000303,0.000489,0.000414,0.000000,0.0,0.000000,0.0,4,0,1,422820.0


In [32]:
full_skater_totals = full_skater_totals.rename(columns = {'Team_x':'Team','Position_EV':'Position'})
full_skater_totals

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,G±_PP,PP_TOI,G±_SH,SH_TOI,G,A1,A2,TOI_tot
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,0.000000,0.0,0.000000,0.0,1,0,1,254520.0
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,0.001642,345240.0,-0.002825,485532.0,5,23,13,5536512.0
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,0.002478,477900.0,-0.001350,311292.0,11,5,6,3161448.0
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,0.002539,769968.0,-0.001128,601200.0,15,13,29,5470920.0
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,0.000000,0.0,0.000000,0.0,0,1,0,1082988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,0.000000,0.0,-0.001653,770652.0,5,2,7,5142528.0
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,0.000000,0.0,-0.001847,524592.0,2,4,4,3622572.0
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,0.000000,0.0,-0.000756,444528.0,2,8,4,4854420.0
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,0.000000,0.0,-0.002558,532332.0,12,4,3,3426372.0


In [33]:
#Look at positions to get correct F/D average points
forwards_table = full_skater_totals.loc[full_skater_totals.Position!= "D"]
defense_table = full_skater_totals.loc[full_skater_totals.Position== "D"]

In [34]:
#Calculate Mean Stats of forwards and defensemen
avg_GF_EV_F = forwards_table['GF_EV'].mean()
avg_GA_EV_F = forwards_table['GA_EV'].mean()
avg_xGF_EV_F = forwards_table['xGF_EV'].mean()
avg_xGA_EV_F = forwards_table['xGA_EV'].mean()
avg_G_PP_F = forwards_table['G±_PP'].mean()
avg_G_SH_F = forwards_table['G±_SH'].mean()
avg_G_F = forwards_table['G'].mean()
avg_A1_F = forwards_table['A1'].mean()
avg_A2_F = forwards_table['A2'].mean()
#print(Forwards_mean)
avg_GF_EV_D = defense_table['GF_EV'].mean()
avg_GA_EV_D = defense_table['GA_EV'].mean()
avg_xGF_EV_D = defense_table['xGF_EV'].mean()
avg_xGA_EV_D = defense_table['xGA_EV'].mean()
avg_G_PP_D = defense_table['G±_PP'].mean()
avg_G_SH_D = defense_table['G±_SH'].mean()
avg_G_D = defense_table['G'].mean()
avg_A1_D = defense_table['A1'].mean()
avg_A2_D = defense_table['A2'].mean()
#print(Defensemen_mean)

In [35]:
#Add impact for all variables to the tables
pd.options.mode.chained_assignment = None
#forwards
forwards_table['GF_Impact'] = (forwards_table['GF_EV']-avg_GF_EV_F)*forwards_table['EV_TOI']
forwards_table['GA_Impact'] = (forwards_table['GA_EV']-avg_GA_EV_F)*forwards_table['EV_TOI']
forwards_table['xGF_Impact'] = (forwards_table['xGF_EV']-avg_xGF_EV_F)*forwards_table['EV_TOI']
forwards_table['xGA_Impact'] = (forwards_table['xGA_EV']-avg_xGA_EV_F)*forwards_table['EV_TOI']
forwards_table['G±_PP_Impact'] = (forwards_table['G±_PP']-avg_G_PP_F)*forwards_table['PP_TOI']
forwards_table['G±_SH_Impact'] = (forwards_table['G±_SH']-avg_G_SH_F)*forwards_table['SH_TOI']
forwards_table['G_Impact'] = (forwards_table['G']-avg_G_F)*forwards_table['TOI_tot']
forwards_table['A1_Impact'] = (forwards_table['A1']-avg_A1_F)*forwards_table['TOI_tot']
forwards_table['A2_Impact'] = (forwards_table['A2']-avg_A2_F)*forwards_table['TOI_tot']
#defense
defense_table['GF_Impact'] = (defense_table['GF_EV']-avg_GF_EV_D)*defense_table['EV_TOI']
defense_table['GA_Impact'] = (defense_table['GA_EV']-avg_GA_EV_D)*defense_table['EV_TOI']
defense_table['xGF_Impact'] = (defense_table['xGF_EV']-avg_xGF_EV_D)*defense_table['EV_TOI']
defense_table['xGA_Impact'] = (defense_table['xGA_EV']-avg_xGA_EV_D)*defense_table['EV_TOI']
defense_table['G±_PP_Impact'] = (defense_table['G±_PP']-avg_G_PP_D)*defense_table['PP_TOI']
defense_table['G±_SH_Impact'] = (defense_table['G±_SH']-avg_G_SH_D)*defense_table['SH_TOI']
defense_table['G_Impact'] = (defense_table['G']-avg_G_D)*defense_table['TOI_tot']
defense_table['A1_Impact'] = (defense_table['A1']-avg_A1_D)*defense_table['TOI_tot']
defense_table['A2_Impact'] = (defense_table['A2']-avg_A2_D)*defense_table['TOI_tot']

forwards_table

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,TOI_tot,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,G±_PP_Impact,G±_SH_Impact,G_Impact,A1_Impact,A2_Impact
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,...,254520.0,15.650529,2.229453,25.494204,-35.882065,-0.000000,0.000000,-2.399057e+06,-2.122005e+06,-1.207542e+06
8,Adam Brooks,19-20,TOR,C,7,185760.0,0.000950,0.000319,0.000767,0.000653,...,197892.0,55.838985,-68.543072,22.284830,-2.545257,-0.000000,0.000000,-2.063184e+06,-1.254097e+06,-9.388766e+05
9,Adam Brooks,20-21,TOR,C,11,397368.0,0.000775,0.000303,0.000489,0.000414,...,422820.0,49.908413,-153.246527,-62.709465,-100.371479,-0.000000,0.000000,-2.716961e+06,-3.525169e+06,-2.006023e+06
10,Adam Brooks,21-22,MTL/VGK/WPG,C,25,768636.0,0.000406,0.000231,0.000572,0.000683,...,805320.0,-187.429567,-351.940087,-57.247036,12.954360,-0.000000,0.000000,-6.785474e+06,-5.908860e+06,-4.626071e+06
11,Adam Erne,19-20,DET,L,56,2158992.0,0.000242,0.000833,0.000483,0.000658,...,2382048.0,-880.298479,312.841470,-352.709390,-17.587799,-225.588907,0.000000,-2.007068e+07,-1.271366e+07,-1.368341e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,Zack MacEwen,20-21,VAN,C,34,1039140.0,0.000589,0.000628,0.000572,0.000631,...,1057212.0,-62.882151,-63.027915,-77.393831,-37.330147,-0.000000,0.000000,-9.965080e+06,-8.814274e+06,-5.015825e+06
2473,Zack MacEwen,21-22,PHI,C,75,2548188.0,0.000475,0.000778,0.000572,0.000864,...,2609100.0,-444.410448,227.670607,-189.785816,503.035894,-0.000000,0.000000,-1.937468e+07,-1.392550e+07,-7.160385e+06
2474,Zack Smith,19-20,CHI,C,50,1850832.0,0.000417,0.000517,0.000469,0.000733,...,2119068.0,-430.754993,-317.908217,-328.072024,123.734966,-0.000000,57.981808,-1.361673e+07,-7.071927e+06,-7.934615e+06
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,...,3426372.0,-461.726097,-312.887239,-304.851557,-329.360143,-0.000000,-965.854758,5.393759e+06,-1.486114e+07,-9.403298e+06


In [36]:
#Show defense table
defense_table

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,TOI_tot,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,G±_PP_Impact,G±_SH_Impact,G_Impact,A1_Impact,A2_Impact
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,...,5536512.0,1241.828544,273.983227,165.122527,86.463107,370.223264,-801.705154,9.575132e+06,9.641494e+07,3.744098e+07
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,...,3161448.0,-0.539512,-512.924836,100.946289,-240.607959,912.060693,-54.846263,2.443626e+07,-1.851396e+06,-7.506635e+05
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,...,5470920.0,1627.261926,-310.311062,1125.744563,-133.125890,1516.519070,27.675103,6.417089e+07,4.056350e+07,1.245321e+08
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,...,1082988.0,67.054361,33.953097,-35.058811,-79.544584,-0.000000,0.000000,-3.541964e+06,-4.966168e+06,-6.755076e+06
5,Adam Boqvist,19-20,CHI,D,41,2030220.0,0.000919,0.000825,0.000764,0.000819,...,2394432.0,512.711666,223.339651,220.049942,292.223954,206.282102,0.000000,1.746623e+06,-1.402219e+06,-5.357405e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,Zach Whitecloud,20-21,VGK,D,51,2861352.0,0.000603,0.000439,0.000667,0.000569,...,3265992.0,-183.489097,-790.029304,31.947046,-303.483326,-0.000000,-18.077949,-4.149599e+06,-5.178611e+06,-7.754867e+05
2467,Zach Whitecloud,21-22,VGK,D,59,3576132.0,0.000928,0.000675,0.000772,0.000622,...,3993048.0,932.917312,-143.018064,417.408177,-190.554674,-0.000000,-589.482289,1.888493e+07,9.640749e+06,-1.292726e+07
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,...,5142528.0,619.733260,-929.900712,-234.012712,-353.499178,-0.000000,-369.116796,8.893756e+06,-1.843913e+07,3.921471e+06
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,...,3622572.0,138.261554,-125.860285,-212.370568,-393.854605,-0.000000,-353.266201,-4.602651e+06,-5.744010e+06,-8.105298e+06


In [37]:
#Add dataframes back together and sort again
concat_frames = [forwards_table, defense_table]
skater_stats = pd.concat(concat_frames)
skater_stats = skater_stats.sort_index()
skater_stats

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,TOI_tot,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,G±_PP_Impact,G±_SH_Impact,G_Impact,A1_Impact,A2_Impact
0,A.J. Greer,21-22,N.J,L,9,253620.0,0.000711,0.000697,0.000747,0.000525,...,254520.0,15.650529,2.229453,25.494204,-35.882065,-0.000000,0.000000,-2.399057e+06,-2.122005e+06,-1.207542e+06
1,Aaron Ekblad,19-20,FLA,D,67,4565808.0,0.000939,0.000775,0.000692,0.000694,...,5536512.0,1241.828544,273.983227,165.122527,86.463107,370.223264,-801.705154,9.575132e+06,9.641494e+07,3.744098e+07
2,Aaron Ekblad,20-21,FLA,D,35,2268540.0,0.000667,0.000489,0.000700,0.000569,...,3161448.0,-0.539512,-512.924836,100.946289,-240.607959,912.060693,-54.846263,2.443626e+07,-1.851396e+06,-7.506635e+05
3,Aaron Ekblad,21-22,FLA,D,61,3933900.0,0.001081,0.000636,0.000942,0.000642,...,5470920.0,1627.261926,-310.311062,1125.744563,-133.125890,1516.519070,27.675103,6.417089e+07,4.056350e+07,1.245321e+08
4,Aaron Ness,19-20,ARI,D,24,1053468.0,0.000731,0.000747,0.000622,0.000600,...,1082988.0,67.054361,33.953097,-35.058811,-79.544584,-0.000000,0.000000,-3.541964e+06,-4.966168e+06,-6.755076e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,4216320.0,0.000814,0.000494,0.000600,0.000592,...,5142528.0,619.733260,-929.900712,-234.012712,-353.499178,-0.000000,-369.116796,8.893756e+06,-1.843913e+07,3.921471e+06
2476,Zdeno Chara,20-21,WSH,D,55,2942712.0,0.000714,0.000672,0.000583,0.000542,...,3622572.0,138.261554,-125.860285,-212.370568,-393.854605,-0.000000,-353.266201,-4.602651e+06,-5.744010e+06,-8.105298e+06
2477,Zdeno Chara,21-22,NYI,D,72,4306680.0,0.000675,0.000592,0.000656,0.000769,...,4854420.0,34.864771,-531.124425,0.232159,404.556977,-0.000000,185.926204,-6.167773e+06,1.172043e+07,-1.086149e+07
2478,Zemgus Girgensons,19-20,BUF,C,69,2827620.0,0.000486,0.000578,0.000539,0.000550,...,3426372.0,-461.726097,-312.887239,-304.851557,-329.360143,-0.000000,-965.854758,5.393759e+06,-1.486114e+07,-9.403298e+06


In [38]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Goalie%20Stats.csv')
goalie_stats_raw.head()

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx
0,Aaron Dell,19-20,S.J,G,33,1834.23,91.06,994.37,1352.10,92.16,90.84,93.27,93.18,0.08,-1.20,1.10
1,Aaron Dell,20-21,N.J,G,7,319.17,21.45,153.29,210.23,11.91,86.01,89.80,94.33,-4.54,-7.35,-9.54
2,Aaron Dell,21-22,BUF,G,12,565.38,37.95,356.65,467.99,30.60,89.36,91.89,93.46,-1.57,-4.85,-7.35
3,Adam Huska,21-22,NYR,G,1,59.70,7.53,42.56,55.97,3.50,82.31,86.55,93.75,-7.20,-3.58,-4.03
4,Adam Werner,19-20,COL,G,2,87.83,4.85,54.22,65.83,3.76,91.05,92.63,94.29,-1.66,0.05,-1.09


In [39]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','GSAx']]
goalie_stats.head()

,Player,Season,Team,Position,GP,GSAx
0,Aaron Dell,19-20,S.J,G,33,1.10
1,Aaron Dell,20-21,N.J,G,7,-9.54
2,Aaron Dell,21-22,BUF,G,12,-7.35
3,Adam Huska,21-22,NYR,G,1,-4.03
4,Adam Werner,19-20,COL,G,2,-1.09


In [40]:
#Add Active / Inactive Column to both data tables
skater_stats['A/I'] = 'Active'
goalie_stats['A/I'] = 'Active'

In [41]:
#List of team names
#print(full_player_totals['Team'].unique())

In [42]:
#Put Players on current teams

#NHL API info
#https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
#https://github.com/dword4/nhlapi
#https://gitlab.com/dword4/nhlapi

#API JSON project example
#https://github.com/danmartin25/World_Weather_Analysis/blob/main/WeatherPy.ipynb

#Test all
#data = requests.get('https://statsapi.web.nhl.com/api/v1/configurations').json()
#data

#Different options
#data = requests.get('https://statsapi.web.nhl.com/api/v1/expands').json()
#data

#Empty Dictionary to Hold Rosters
#active_rosters = []

### NOTE: ONLY RUN REQUEST ONCE, THEN COMMENT OUT. INFO WILL STAY IN VARIABLE
#json_api_data = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster').json()

#Variable where data is stored
#json_api_data

#Devils Roster (Index 0, change indext to 1, 2, etc for other teams rosters)
#json_api_data["teams"][0]

#player_name = json_api_data["teams"][0]
#player_team = 
#player_position = #code


#active_rosters.append{{"Player": player_name,
#                       "Team": player_team,
#                       "Postion": player_position}}




In [43]:
#Make a Data Table for skaters on each team
Avalanche_skaters = skater_stats.loc[(skater_stats.Team == 'COL')]
Blackhawks_skaters = skater_stats.loc[(skater_stats.Team == 'CHI')]
Blue_Jackets_skaters = skater_stats.loc[(skater_stats.Team == 'CBJ')]
Blues_skaters = skater_stats.loc[(skater_stats.Team == 'STL')]
Bruins_skaters = skater_stats.loc[(skater_stats.Team == 'BOS')]
Canadiens_skaters = skater_stats.loc[(skater_stats.Team == 'MTL')]
Canucks_skaters = skater_stats.loc[(skater_stats.Team == 'VAN')]
Capitals_skaters = skater_stats.loc[(skater_stats.Team == 'WSH')]
Coyotes_skaters = skater_stats.loc[(skater_stats.Team == 'ARI')]
Devils_skaters = skater_stats.loc[(skater_stats.Team == 'N.J')]
Ducks_skaters = skater_stats.loc[(skater_stats.Team == 'ANA')]
Flames_skaters = skater_stats.loc[(skater_stats.Team == 'CGY')]
Flyers_skaters = skater_stats.loc[(skater_stats.Team == 'PHI')]
Hurricanes_skaters = skater_stats.loc[(skater_stats.Team == 'CAR')]
Islanders_skaters = skater_stats.loc[(skater_stats.Team == 'NYI')]
Jets_skaters = skater_stats.loc[(skater_stats.Team == 'WPG')]
Kings_skaters = skater_stats.loc[(skater_stats.Team == 'L.A')]
Knights_skaters = skater_stats.loc[(skater_stats.Team == 'VGK')]
Kraken_skaters = skater_stats.loc[(skater_stats.Team == 'SEA')]
Leafs_skaters = skater_stats.loc[(skater_stats.Team == 'TOR')]
Lightning_skaters = skater_stats.loc[(skater_stats.Team == 'T.B')]
Oilers_skaters = skater_stats.loc[(skater_stats.Team == 'EDM')]
Panthers_skaters = skater_stats.loc[(skater_stats.Team == 'FLA')]
Penguins_skaters = skater_stats.loc[(skater_stats.Team == 'PIT')]
Predators_skaters = skater_stats.loc[(skater_stats.Team == 'NSH')]
Rangers_skaters = skater_stats.loc[(skater_stats.Team == 'NYR')]
Red_Wings_skaters = skater_stats.loc[(skater_stats.Team == 'DET')]
Sabres_skaters = skater_stats.loc[(skater_stats.Team == 'BUF')]
Senators_skaters = skater_stats.loc[(skater_stats.Team == 'OTT')]
Sharks_skaters = skater_stats.loc[(skater_stats.Team == 'S.J')]
Stars_skaters = skater_stats.loc[(skater_stats.Team == 'DAL')]
Wild_skaters = skater_stats.loc[(skater_stats.Team == 'MIN')]
Avalanche_skaters.head(10)

,Player,Season,Team,Position,GP_EV,EV_TOI,GF_EV,GA_EV,xGF_EV,xGA_EV,...,GF_Impact,GA_Impact,xGF_Impact,xGA_Impact,G±_PP_Impact,G±_SH_Impact,G_Impact,A1_Impact,A2_Impact,A/I
84,Alex Newhook,20-21,COL,C,6,268920.0,0.001397,0.000211,0.000978,0.000467,...,201.103670,-128.361052,89.033179,-53.733703,-0.000000,0.000000,-3.080327e+06,-2.167815e+06,-1.106287e+06,Active
85,Alex Newhook,21-22,COL,C,71,3072672.0,0.000669,0.000425,0.000631,0.000631,...,61.582213,-809.439204,-49.609520,-110.382911,176.340056,0.000000,8.924849e+06,9.231782e+06,1.128737e+07,Active
125,Andre Burakovsky,19-20,COL,L,58,2608200.0,0.001000,0.000644,0.000700,0.000711,...,914.428307,-114.727556,139.014567,116.407818,151.791697,0.000000,3.037136e+07,1.796334e+07,1.667191e+07,Active
126,Andre Burakovsky,20-21,COL,L,53,2430792.0,0.000917,0.000756,0.000767,0.000508,...,649.663512,163.164135,291.611690,-384.420763,696.945619,0.000000,2.468996e+07,1.918575e+07,1.225432e+07,Active
127,Andre Burakovsky,21-22,COL,L,80,4044168.0,0.001047,0.000858,0.000786,0.000714,...,1608.849648,687.110755,563.798088,191.730991,536.573488,0.000000,5.421929e+07,7.337200e+07,4.335791e+07,Active
261,Bowen Byram,20-21,COL,D,19,1115388.0,0.000658,0.000492,0.000642,0.000519,...,-9.560165,-249.094832,-15.431373,-174.070700,-0.000000,0.000000,-3.917436e+06,-5.492615e+06,-6.273367e+06,Active
262,Bowen Byram,21-22,COL,D,30,1879812.0,0.000925,0.000756,0.000717,0.000686,...,485.171037,76.251126,114.978734,19.933065,-7.634087,0.000000,3.512911e+06,8.417075e+05,-4.823007e+05,Active
294,Brandon Saad,20-21,COL,L,44,1915020.0,0.000978,0.000903,0.000761,0.000550,...,628.845157,410.477029,219.097735,-223.060829,181.566704,0.000000,1.020136e+07,-1.413341e+07,2.800264e+06,Active
382,Cale Makar,19-20,COL,D,57,3436560.0,0.001058,0.000661,0.000717,0.000614,...,1345.168705,-185.166247,210.197253,-211.755562,1046.804369,0.000000,3.763581e+07,7.076832e+07,4.208990e+07,Active
383,Cale Makar,20-21,COL,D,44,2964420.0,0.001083,0.000617,0.000794,0.000500,...,1234.469991,-291.478740,411.884802,-520.277518,1420.122413,111.502479,1.821053e+07,6.320280e+07,2.988935e+07,Active


In [44]:
#Make a Data for each team of goalies
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Team == 'COL')]
Blackhawks_goalie = goalie_stats.loc[(goalie_stats.Team == 'CHI')]
Blue_Jackets_goalie = goalie_stats.loc[(goalie_stats.Team == 'CBJ')]
Blues_goalie = goalie_stats.loc[(goalie_stats.Team == 'STL')]
Bruins_goalie = goalie_stats.loc[(goalie_stats.Team == 'BOS')]
Canadiens_goalie = goalie_stats.loc[(goalie_stats.Team == 'MTL')]
Canucks_goalie = goalie_stats.loc[(goalie_stats.Team == 'VAN')]
Capitals_goalie = goalie_stats.loc[(goalie_stats.Team == 'WSH')]
Coyotes_goalie = goalie_stats.loc[(goalie_stats.Team == 'ARI')]
Devils_goalie = goalie_stats.loc[(goalie_stats.Team == 'N.J')]
Ducks_goalie = goalie_stats.loc[(goalie_stats.Team == 'ANA')]
Flames_goalie = goalie_stats.loc[(goalie_stats.Team == 'CGY')]
Flyers_goalie = goalie_stats.loc[(goalie_stats.Team == 'PHI')]
Hurricanes_goalie = goalie_stats.loc[(goalie_stats.Team == 'CAR')]
Islanders_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYI')]
Jets_goalie = goalie_stats.loc[(goalie_stats.Team == 'WPG')]
Kings_goalie = goalie_stats.loc[(goalie_stats.Team == 'L.A')]
Knights_goalie = goalie_stats.loc[(goalie_stats.Team == 'VGK')]
Kraken_goalie = goalie_stats.loc[(goalie_stats.Team == 'SEA')]
Leafs_goalie = goalie_stats.loc[(goalie_stats.Team == 'TOR')]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Team == 'T.B')]
Oilers_goalie = goalie_stats.loc[(goalie_stats.Team == 'EDM')]
Panthers_goalie = goalie_stats.loc[(goalie_stats.Team == 'FLA')]
Penguins_goalie = goalie_stats.loc[(goalie_stats.Team == 'PIT')]
Predators_goalie = goalie_stats.loc[(goalie_stats.Team == 'NSH')]
Rangers_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYR')]
Red_Wings_goalie = goalie_stats.loc[(goalie_stats.Team == 'DET')]
Sabres_goalie = goalie_stats.loc[(goalie_stats.Team == 'BUF')]
Senators_goalie = goalie_stats.loc[(goalie_stats.Team == 'OTT')]
Sharks_goalie = goalie_stats.loc[(goalie_stats.Team == 'S.J')]
Stars_goalie = goalie_stats.loc[(goalie_stats.Team == 'DAL')]
Wild_goalie = goalie_stats.loc[(goalie_stats.Team == 'MIN')]

In [45]:
Avalanche_goalie

,Player,Season,Team,Position,GP,GSAx,A/I
4,Adam Werner,19-20,COL,G,2,-1.09,Active
28,Antoine Bibeau,19-20,COL,G,2,-0.57,Active
93,Darcy Kuemper,21-22,COL,G,57,15.77,Active
117,Hunter Miska,20-21,COL,G,5,-9.33,Active
175,Justus Annunen,21-22,COL,G,2,-2.65,Active
231,Pavel Francouz,19-20,COL,G,34,2.68,Active
232,Pavel Francouz,21-22,COL,G,21,-0.10,Active
239,Philipp Grubauer,19-20,COL,G,36,-1.71,Active
240,Philipp Grubauer,20-21,COL,G,40,4.79,Active


**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [ ]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_skater_totals.columns

Index(['Player', 'Season', 'Team', 'Position', 'GP', 'TOI_EV', 'G_EV', 'xG_EV',
       'TOI_PP', 'GAA_PP', 'TOI_SH', 'GAA_SH', 'TOI_tot', 'Points', 'PAA',
       'A/I'],
      dtype='object')

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_skater_totals dataframe and the goalie_stats dataframe.

In [ ]:
#Import Skater Box Car Stats
skater_box = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
skater_box

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,...,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,...,20,11,1,6,0,0,5,2,1,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,...,34,56,14,10,0,0,-4,0,0,0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,...,24,46,14,8,1,1,-6,0,0,0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,...,62,54,13,9,0,0,-4,0,0,0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,...,17,49,0,5,0,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,1428.48,5,2,7,14,...,78,67,20,10,4,4,-10,0,0,0
2493,Zdeno Chara,20-21,WSH,D,55,1006.27,2,4,4,10,...,88,33,12,4,2,2,-8,0,0,0
2494,Zdeno Chara,21-22,NYI,D,72,1348.45,2,8,4,14,...,125,46,20,6,5,5,-14,0,0,0
2495,Zemgus Girgensons,19-20,BUF,C,69,951.77,12,4,3,19,...,110,71,5,6,0,0,1,30,43,-13


In [ ]:
#Restrict Data Table to Player, Season, Team, G, A1, A2, iSH, iBLK, iPEN±, FO±
skater_box = skater_box.loc[:,['Player','Season','Team','Position','GP','G','A1','A2','iSF','iBLK','iPEN±','FO±']]
skater_box

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0
2493,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0
2494,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0
2495,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13


In [ ]:
#Import EV Skater Data
skater_EV = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Totals.csv')
skater_EV

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,53.16,53.30,53.14,...,1070.09,52.60,52.78,12.38,84.53,114.87,143.58,-0.18,10.03,90.59
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,54.51,53.65,53.97,...,481.39,26.51,21.58,6.70,54.32,59.26,83.07,4.93,7.68,93.24
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,58.90,59.83,59.35,...,857.55,61.71,41.78,29.17,214.91,307.62,394.70,19.93,9.96,91.61
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,51.47,49.69,49.55,...,276.12,10.93,10.53,-0.29,8.70,-2.64,-4.93,0.40,8.39,90.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,62.20,49.10,48.67,48.25,...,1085.01,42.25,41.58,22.43,-21.33,-43.04,-73.23,0.67,9.85,94.23
2476,Zdeno Chara,20-21,WSH,D,55,817.42,51.49,49.37,49.02,50.39,...,706.99,28.61,26.57,2.03,-9.96,-21.06,11.25,2.04,9.04,91.70
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,53.26,44.88,44.56,43.73,...,1246.03,46.96,55.16,5.92,-123.65,-180.44,-277.79,-8.20,8.92,93.63
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,45.67,47.49,49.21,49.34,...,621.61,25.45,25.86,-4.34,-35.51,-15.13,-16.25,-0.41,6.82,92.66


In [ ]:
#Restrict Data to Goal Differential and Corsi Differential
skater_EV = skater_EV.loc[:,['Player', 'Season', 'Position','G±', 'C±']]
skater_EV

,Player,Season,Position,G±,C±
0,A.J. Greer,21-22,L,0.06,17.36
1,Aaron Ekblad,19-20,D,12.38,143.58
2,Aaron Ekblad,20-21,D,6.70,83.07
3,Aaron Ekblad,21-22,D,29.17,394.70
4,Aaron Ness,19-20,D,-0.29,-4.93
...,...,...,...,...,...
2475,Zdeno Chara,19-20,D,22.43,-73.23
2476,Zdeno Chara,20-21,D,2.03,11.25
2477,Zdeno Chara,21-22,D,5.92,-277.79
2478,Zemgus Girgensons,19-20,C,-4.34,-16.25


In [ ]:
#Merge EV and box car stats together
Game_Score = pd.merge(skater_box, skater_EV)
Game_Score

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0,12.38,143.58
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0,6.70,83.07
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0,29.17,394.70
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0,-0.29,-4.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0,22.43,-73.23
2476,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0,2.03,11.25
2477,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0,5.92,-277.79
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25


In [ ]:
#Add column for player's average game score
Game_Score['GS'] = ((Game_Score['G']*0.75) + (Game_Score['A1']*0.7) + (Game_Score['A2']*0.55) + (Game_Score['iSF']*0.075) + (Game_Score['iBLK']*0.05) + (Game_Score['iPEN±']*0.15) + (Game_Score['FO±']*0.01) + (Game_Score['C±']*0.05) + (Game_Score['G±']*0.15))/ Game_Score['GP']
Game_Score

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36,0.459667
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0,12.38,143.58,0.756134
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0,6.70,83.07,0.806671
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0,29.17,394.70,1.258369
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0,-0.29,-4.93,0.124375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0,22.43,-73.23,0.275044
2476,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0,2.03,11.25,0.264400
2477,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0,5.92,-277.79,0.066299
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25,0.280891


In [ ]:
#Separate Data into forwards and defensemen
forwards_GS = Game_Score.loc[Game_Score.Position!="D"]
defense_GS = Game_Score.loc[Game_Score.Position=="D"]

In [ ]:
#Calculate Average Game Score for forwards and defensemen
avg_GS_forwards = forwards_GS['GS'].mean()
avg_GS_defense = defense_GS['GS'].mean()

In [ ]:
#Calculate GSVA for forwards
pd.options.mode.chained_assignment = None
forwards_GS['GSVA'] = forwards_GS['GS']-avg_GS_forwards
forwards_GS

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS,GSVA
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36,0.459667,0.014407
8,Adam Brooks,19-20,TOR,C,7,0,2,1,2,1,1,0,1.95,-2.68,0.351214,-0.094046
9,Adam Brooks,20-21,TOR,C,11,4,0,1,10,2,1,-3,3.13,3.73,0.470545,0.025285
10,Adam Brooks,21-22,MTL/VGK/WPG,C,25,2,1,0,14,6,0,-11,2.24,-29.75,0.091540,-0.353720
11,Adam Erne,19-20,DET,L,56,2,3,0,63,25,-6,7,-21.29,-138.97,-0.024946,-0.470206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,Zack MacEwen,20-21,VAN,C,34,1,0,1,31,9,-2,0,-0.68,-25.54,0.070471,-0.374789
2473,Zack MacEwen,21-22,PHI,C,75,3,3,3,82,32,-9,-3,-13.76,-221.36,-0.010160,-0.455420
2474,Zack Smith,19-20,CHI,C,50,4,5,2,43,18,-7,-14,-3.10,-79.20,0.122200,-0.323060
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25,0.280891,-0.164369


In [ ]:
forwards_GS.loc[forwards_GS.Player=='Auston Matthews']

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS,GSVA
218,Auston Matthews,19-20,TOR,C,70,47,19,14,290,60,10,92,19.58,185.69,1.366307,0.921047
219,Auston Matthews,20-21,TOR,C,52,41,15,10,222,47,9,33,25.06,148.83,1.512125,1.066865
220,Auston Matthews,21-22,TOR,C,73,60,26,20,348,62,4,153,28.36,467.33,1.823979,1.378719


In [ ]:
avg_GS_forwards

0.4452599914524993

Things for next steps
1. Add goalie table.
2. Separate Players to teams.
3. Create matchups
4. Determine Log Loss for Teams
5.